<a href="https://colab.research.google.com/github/fbghkh1999/dataIntroduction/blob/main/gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras

In [ ]:
pip install gym 

In [ ]:
pip install keras-rl

     |████████████████████████████████| 40kB 1.7MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48380 sha256=c07bbf3051cbfc042c1596f735966e630baefd421d9df3a0cc4853377b09a628
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl


In [ ]:
pip install git+git://github.com/wau/keras-rl2.git --upgrade --no-deps

  Cloning git://github.com/wau/keras-rl2.git to /tmp/pip-req-build-zd95sm4e
  Running command git clone -q git://github.com/wau/keras-rl2.git /tmp/pip-req-build-zd95sm4e
  Created wheel for keras-rl2: filename=keras_rl2-1.0.4-cp36-none-any.whl size=53199 sha256=f4fa0ecbf0f11846d78a28faf4a82e2aa89399162e269a6e9ea1393038a25313
  Stored in directory: /tmp/pip-ephem-wheel-cache-3eqxgpdn/wheels/2b/a2/35/5c5d276e34f74ea063a6f2498e9b4a059027fa9da74c7fd6a3
Successfully built keras-rl2


In [ ]:
'''import gym
env = gym.make('CartPole-v0')
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample()) # take a random action
env.close()'''
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
import random
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam
ENV_NAME = 'MountainCar-v0'

# Get the environment and extract the number of actions available in the Cartpole problem
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
env.reset()
goal_steps = 200
score_requirement = -198
intial_games = 10000



def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 3)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            if observation[0] > -0.2:
                reward = 1
            
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1, 0]
                elif data[1] == 0:
                    output = [1, 0, 0]
                elif data[1] == 2:
                    output = [0, 0, 1]
                training_data.append([data[0], output])
        
        env.reset()
    
    print(accepted_scores)
    return training_data

nb_actions = env.action_space.n


def build_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model



def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=5)
    return model
train=model_data_preparation()
trained_model = train_model(train)
'''
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))'''
#print(trai.summary())

[-186.0, -196.0, -178.0, -176.0, -186.0, -174.0, -184.0, -198.0, -176.0, -178.0, -182.0, -174.0, -178.0, -190.0, -180.0, -184.0, -176.0, -168.0, -174.0, -188.0, -174.0, -194.0, -192.0, -176.0, -192.0, -178.0, -176.0, -176.0, -192.0, -172.0, -186.0, -168.0, -176.0, -184.0, -190.0, -188.0, -194.0, -176.0, -196.0, -170.0, -190.0, -170.0]
Train on 8358 samples
Epoch 1/5
8358/8358 [==============================] - 0s 33us/sample - loss: 0.2274
Epoch 2/5
8358/8358 [==============================] - 0s 35us/sample - loss: 0.2218
Epoch 3/5
8358/8358 [==============================] - 0s 33us/sample - loss: 0.2208
Epoch 4/5
8358/8358 [==============================] - 0s 31us/sample - loss: 0.2200
Epoch 5/5
8358/8358 [==============================] - 0s 33us/sample - loss: 0.2197


"\nmodel = Sequential()\nmodel.add(Flatten(input_shape=(1,) + env.observation_space.shape))\nmodel.add(Dense(16))\nmodel.add(Activation('relu'))\nmodel.add(Dense(nb_actions))\nmodel.add(Activation('linear'))"

In [ ]:
'''policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=50000, window_length=1)
dqn = DQNAgent(model=trained_model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
# Okay, now it's time to learn something! We visualize the training here for show, but this slows down training quite a lot. 
dqn.fit(env, nb_steps=5000, visualize=False, verbose=2)'''

"policy = EpsGreedyQPolicy()\nmemory = SequentialMemory(limit=50000, window_length=1)\ndqn = DQNAgent(model=trained_model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,target_model_update=1e-2, policy=policy)\ndqn.compile(Adam(lr=1e-3), metrics=['mae'])\n# Okay, now it's time to learn something! We visualize the training here for show, but this slows down training quite a lot. \ndqn.fit(env, nb_steps=5000, visualize=False, verbose=2)"

In [ ]:
env.seed

<bound method Wrapper.seed of <TimeLimit<MountainCarEnv<MountainCar-v0>>>>